In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import numpy as np
import cv2
from unet_utils import *
import zarr
import glob

import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import load_model

Using TensorFlow backend.


In [2]:
def load_zarr(patient_id):
    lung_mask_zarr = zarr_load_group['lung_mask'][patient_id]
    return np.array(lung_mask_zarr).astype('float32')

def save_zarr(id_patient, cand):
    cand_group.array(id_patient, cand, 
            chunks=(1, 1, 17, 21, 21), compressor=zarr.Blosc(clevel=9, cname="zstd", shuffle=2),
            synchronizer=zarr.ThreadSynchronizer())
    return

zarr_store = zarr.DirectoryStore('/home/w/DS_Projects/Kaggle/DS Bowl 2017/input_data/stage1_3d_zarr/3d.zarr/')
zarr_group = zarr.hierarchy.open_group(store=zarr_store, mode='a')
lung_mask_group = zarr_group.require_group('lung_mask')
cand_group = zarr_group.require_group('candidates')

zarr_load_group = zarr.hierarchy.open_group(store=zarr_store, mode='r')

In [3]:
def load_unet(check_name):
    check_model = '/home/w/DS_Projects/Kaggle/DS Bowl 2017/Scripts/LUNA/CNN/Checkpoints/{}.h5'.format(check_name)
    model = load_model(check_model, custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef})
    return model

def predict_dsbpats(start, end):
    model = load_unet('2DUnet_genfulldata')
    ids = sorted([x for x in os.listdir(zarr_dir + '/lung_mask/') if 'z' not in x])[start:end]
    for i in range(len(ids)):
        lung = load_zarr(ids[i])
        preds = model.predict(lung, batch_size = 8)
        cands = lung * preds
        save_zarr(ids[i], preds, cands)
    return 

def predict_dsb3d(ids):
    model = load_unet('3DUNet_genfulldata_patients_merged_cont')
    masks = np.full((1, 1, 136, 168, 168), 7.).astype('float32')
    for i in range(len(ids)):
        masks = np.full((1, 1, 136, 168, 168), 7.).astype('float32')
        patient_id = ids[i].split('/')[-1][:-4]
        print(patient_id)
        mask = load_zarr(ids[i])
        mask = mask.swapaxes(1, 0)
        num_slices = mask.shape[1]
        offset = (136 - num_slices)
        if offset == 0:
            masks[0, :, :, :, :] = mask[:, :, :, :]
        if offset > 0:
            begin_offset = int(np.round(offset/2))
            end_offset = int(offset - begin_offset)
            masks[0, :, begin_offset:-end_offset, :, :] = mask[:, :, :, :]
        if offset < 0:
            offset = -(136 - num_slices)
            begin_offset = int(np.round(offset/2))
            end_offset = int(offset - begin_offset)
            masks[0, :, :, :, :] = mask[:, begin_offset:-end_offset, :, :]
        preds = (model.predict(masks, batch_size = 1)).astype('float32')
        thres = 0.9
        preds[preds <= thres] = 0.
        preds[preds > thres] = 1.
        cands = masks * preds
        save_zarr(patient_id, cands)
    return

dsbpats = [x for x in os.listdir('/home/w/DS_Projects/Kaggle/DS Bowl 2017/input_data/stage1_3d_zarr/3d.zarr/lung_mask/') if '.npy' in x]

predict_dsb3d(dsbpats)

0c98fcb55e3f36d0c2b6507f62f4c5f1
b0d5ba3a5ac7998791f7e484c070dcb9
916ab7b22affc5651abfc9aae01bb355
9ce92cb785c9878389bd5f765a07e1ae
aa55708fcc8bf27b605bcd2fca0dc991
8e60f166f1f1dc0d72f997fe1c9e72b4
8ed68f2dbf103a4bc0fd8708d8c1ac93
7ce310b8431ace09a91ededcc03f7361
264e5829ef58570c243bfd961098343f
39ebb8121ea6faec0405a4e8db883b55
c610439ebef643c7fd4b30de8088bb55
3d521ee3776eefb65953f89c0e1a4f96
c3e23e84b9f9f31c3ba5458c7f43ec52
9560214767e1257dc97c5103a27f1d2c
19ca60da880961c50f1b99b66f252e8f
ce0609052ba7b533e2974728b0c76bfe
89bfbba58ee5cd0e346cdd6ffd3fa3a3
b06ccf39536d0e3e0e8c78ba599ed23b
8e9002a485cbda2b47cd14014d6f1c36
5368521797323c89292781b8780e45b5
fe45462987bacc32dbc7126119999392
56995718fdd0a0549d0e7f11eb307f34
42dbd247a9e5f710926b7c049a00ff48
b6687898fe385b68d5ae341419ef3fdd
f207bb8bcccdc76143380849402a4f21
caddd8f856b110ed9bb52872e003193f
3f057e68a02e9951b268227649f4a7d6
f6a2e171a3be579b564cd855985ab9d1
75aef267ad112a21c870b7e2893aaf8a
9842a3b9dfabab4ae8642d9953058ce4
f44dd045f6

In [ ]:
preds2 = pr.copy()

thres = 0.9
preds2[preds2 <= thres] = 0.
preds2[preds2 > thres] = 1.
print_mask(ms[0][0][64], preds2[0][0][64])